In [ ]:
# !pip install opencv-python

In [ ]:
import tensorflow as tf # this is to import tensorflow
import json # this is to import json
import pandas as pd # this is to import pandas
import numpy as np # this is to import numpy
from skimage import io, transform # this is to import skimage
from sklearn.model_selection import train_test_split # this is just for train test split lol
import cv2 # compooter vision
from tensorflow.keras import datasets, layers, models, losses, Model # this imports things
from tqdm.notebook import tqdm # this is literally just a progress bar
import os # operating system things
import glob
from PIL import Image # this is to resize images
import random # this is to randomize the images

In [ ]:
# Read the json
with open("/kaggle/input/amogus2/amogus2.json", 'r') as f:
    data_df = pd.json_normalize([i for i in json.loads(f.read()) if i is not None])
data_df = data_df.set_index("id")
data_df

In [ ]:
value_counts = data_df['location_id'].value_counts()

# create a boolean mask to select the rows where the value occurs >= 10 times
mask = data_df['location_id'].isin(value_counts[value_counts >= 10].index)

# apply the boolean mask to the dataframe to filter out the rows
df_filtered = data_df[mask]
df_filtered

In [ ]:
imagestr = glob.glob("/kaggle/input/imgsix/img2/*.jpg")
random.shuffle(imagestr)
images = {file: cv2.imread(file) for file in tqdm(imagestr[:21000])}

In [ ]:
img_series = pd.Series(images)
img_series = img_series[img_series.notna()].copy()
img_series = img_series[[i.shape==(192, 256, 3) for i in tqdm(img_series)]].copy() # change to reshape
# img_series = img_series.apply(lambda x: reshape(x))
img_series = img_series.rename(lambda str: str[26:-4]) # convert file name e.g. "img2\1000148914149750.jpg"->1000148914149750
img_series

In [ ]:
data_df["img"] = img_series
data_df = data_df.dropna()
data_df

In [ ]:
country_df = data_df[data_df["country"] != "null"]
country_df

In [ ]:
country_df.groupby(['country']).count().sort_values(by='img')

In [ ]:
wts = dict(1 / (1 + np.log(country_df.groupby(['country']).count().reset_index()["img"])))
# wts = dict(1 / country_df.groupby(['country']).count().reset_index()["img"])
print(wts)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown='ignore')
enc_country_df = pd.DataFrame(encoder.fit_transform(country_df[['country']]).toarray())
enc_country_df

In [ ]:
X_train_index, X_test_index, y_train, y_test = train_test_split(list(country_df.index), enc_country_df, test_size=0.20)

In [ ]:
X_train = np.stack(img_series[X_train_index].values)
X_test = np.stack(img_series[X_test_index].values)

In [ ]:
X_train

In [ ]:
resnet_model = tf.keras.applications.ResNet152(weights = 'imagenet', include_top = False, input_shape = (192,256,3))
unfreeze_layers = 5
for layer in resnet_model.layers[:-unfreeze_layers]:
    layer.trainable = False

In [ ]:
model = models.Sequential()
#model.add(layers.Lambda(lambda image: tf.image.resize(image, (128, 256))))
model.add(resnet_model)
model.add(layers.Flatten())
model.add(layers.Dense(1000, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.BatchNormalization())
model.add(layers.Dense(len(enc_country_df.columns), activation="softmax"))

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, batch_size=64, epochs=20, class_weight=wts, validation_data=(X_test[:100], y_test[:100]))

In [ ]:
# from collections import Counter
# Counter([i.shape for i in tqdm(img_series)])

In [ ]:
# theimg = io.imread("random_images/boarding.jpg") #country_df['image'][380]
# reshaped = reshape()
# io.imshow(reshaped)

In [ ]:
# p = model.predict(reshaped[None, ...])
# argmax = p[0].argmax()
# encoder.categories_[0][argmax]

In [ ]:
!mkdir saved_model

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("hug")

In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import login
login(token = secret_value_0)

In [ ]:
model.save("saved_model/model3")

In [ ]:
try:
    model.load("saved_model/model3")
except:
    print("error loading model")

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

# Upload all the content from the local folder to your remote Space.
# By default, files are uploaded at the root of the repo
api.upload_folder(
    folder_path="saved_model",
    path_in_repo="model_is_here_3",
    repo_id="hemidemisemipresent/kabirj",
    repo_type="model",
)

In [ ]:
print(encoder.categories_[0])